<a href="https://colab.research.google.com/github/TaeWoo1/goorm/blob/main/260107.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab에서는 번거로울 것 같아 vscode에서...

In [ ]:
!pip list | grep requests
!pip list | grep -i BeautifulSoup4

requests                2.32.5
beautifulsoup4          4.14.2


### 웹 크롤링(bs4, requests), 웹 자동화(selenium), 메일 자동화/REST API


#### 파이썬 웹 크롤링 및 자동화 - bs4, requests

##### 웹 페이지 가져오기


네이버 날씨는 JS라서 request로 안되므로 위키백과를 사용

In [ ]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}



html = requests.get('https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8C%80%EB%AC%B8',  headers=headers)
# pprint(html.text)

##### 요소 1개 찾기(find)

In [ ]:
# 웹 페이지는 HTML로 써져있기 때문에 파싱을 통해 각 요소에 접근이 쉽게 만듦
soup = bs(html.text, 'html.parser')

In [ ]:
# <p style="font-size: 1rem; margin-top: .1em; margin-bottom: 1em;">우리 모두가 만들어가는 자유 백과사전<span class="nomobile"><br>문서 <a href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%ED%86%B5%EA%B3%84" title="위키백과:통계"><abbr style="text-decoration: none; cursor: pointer;" title="넘겨주기와 막다른 문서를 제외한 모든 글의 수"><span style="color: #246ad3;"><b>734,430</b></span></abbr></a>개와 최근 기여자 <b><a href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%ED%86%B5%EA%B3%84" title="위키백과:통계">6,793</a></b>명</span></p>
# 위키백과 문서 수
data1 = soup.find("abbr").find("b")
pprint(data1)

<b>734,430</b>


##### 요소 모두 찾기(findAll)

find와 사용방법이 똑같으나 find는 처음 매칭한 1개만, findALL은 매칭된 모든 것을 반환하여 리스트로 변환

In [ ]:
# <a href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%82%AC%EB%9E%91%EB%B0%A9" title="위키백과:사랑방">사랑방</a>
import re

data2 = soup.find_all("a", href=re.compile(r"^/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC"))
# pprint(data2)

##### 내부 텍스트 추출

In [ ]:
pprint(data2[0])

<a accesskey="z" href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8C%80%EB%AC%B8" title="대문으로 가기 [z]"><span>대문</span></a>


In [ ]:
find_text = data2[0]
find_text

<a accesskey="z" href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8C%80%EB%AC%B8" title="대문으로 가기 [z]"><span>대문</span></a>

In [ ]:
find_text = data2[0].text
find_text

'대문'

네이버 웹툰도 JS기반이라 생략. JS기반은 BS로 어렵지만 불러오고 나서는 for, 리스트 컴프리헨션 등을 이용해 만들 수 있음

#### selenium

In [ ]:
!chromedriver --version

ChromeDriver 142.0.7444.175 (302067f14a4ea3f42001580e6101fa25ed343445-refs/branch-heads/7444@{#2749})


In [ ]:
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

op = ChromeOptions()
op.add_argument("--headless")
op.add_argument("--no-sandbox")
op.add_argument("window-size=1920,1080")
op.add_argument("disable-gpu")
op.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = service, options = op)

driver.get('https://www.google.com/')
time.sleep(3)
print(driver.title)
driver.quit()



Google


##### 유튜브 키워드 검색

In [ ]:
# //*[@id="center"]/yt-searchbox/div[1]/form/input
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get('https://www.youtube.com/')

time.sleep(3)

search = driver.find_element(By.XPATH, '//*[@id="center"]/yt-searchbox/div[1]/form/input')

search.send_keys('반원 코딩')
time.sleep(1)

The chromedriver version (142.0.7444.175) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (143.0.7499.170); currently, chromedriver 143.0.7499.192 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry


##### Enter 전송

In [ ]:
# //*[@id="center"]/yt-searchbox/div[1]/form/input
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()
driver.get('https://www.youtube.com/')

time.sleep(3)

search = driver.find_element(By.XPATH, '//*[@id="center"]/yt-searchbox/div[1]/form/input')

search.send_keys('반원 코딩')
time.sleep(1)

search.send_keys(Keys.ENTER)

The chromedriver version (142.0.7444.175) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (143.0.7499.170); currently, chromedriver 143.0.7499.192 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry


##### 1to50

https://zzzscore.com/1to50/

In [ ]:
# 1,1 : //*[@id="grid"]/div[1]/span
# 1,2 : //*[@id="grid"]/div[2]/span
# 2.1 : //*[@id="grid"]/div[6]/span

from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/1to50/')
driver.implicitly_wait(300)

btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div[*]/span')
print(len(btns))
print(btns[0].text)
print()


25




###### 1번 누르기

In [ ]:
//*[@id="grid"]/div[1]
//*[@id="grid"]/div[2]
//*[@id="grid"]/div[3]

In [ ]:

from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/1to50/')
driver.implicitly_wait(300)

btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div[*]')

for btn in btns:
    if btn.text == '1':
        print(btn.text)
        btn.click()

T


###### 전체에 대해 순차적으로 처리

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/1to50/')
driver.implicitly_wait(300)


num = 1

def clickBtn():
    global num
    btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div[*]')

    for btn in btns:
        print(btn.text, end = '\t')
        if btn.text == str(num):
            btn.click()
            print(True)
            num += 1
            return

while num <= 50:
    clickBtn()

18	13	8	19	11	20	22	12	6	25	10	1	True
18	13	8	19	11	20	22	12	6	25	10	1	2	True
18	13	8	19	11	20	22	12	6	25	10	28	2	3	True
18	13	8	19	11	20	22	12	6	25	10	28	29		9	15	24	16	23	14	5	21	17	7	4	True
18	13	8	19	11	20	22	12	6	25	10	28	29	37	9	15	24	16	23	14	5	True
18	13	8	19	11	20	22	12	6	True
18	13	8	19	11	20	22	12	6	25	10	28	29	37	9	15	24	16	23	14	38	21	17	7	True
18	13	8	True
18	13	8	19	11	20	22	12	35	25	10	28	29	37	9	True
18	13	31	19	11	20	22	12	35	25	10	True
18	13	31	19	11	True
18	13	31	19	11	20	22	12	True
18	13	True
18	13	31	19	44	20	22	42	35	25	43	28	29	37	48	15	24	16	23	14	True
18	39	31	19	44	20	22	42	35	25	43	28	29	37	48	15	True
18	39	31	19	44	20	22	42	35	25	43	28	29	37	48		24	16	True
18	39	31	19	44	20	22	42	35	25	43	28	29	37	48	36	24		23	49	38	21	17	True
18	True
18	39	31	19	True
18	39	31	19	44	20	True
	39	31	19	44	20	22	42	35	25	43	28	29	37	48	36	24	45	23	49	38	21	True
46	39	31	32	44	27	22	True
46	39	31	32	44	27	22	42	35	25	43	28	29	37	48	36	24	45	23	True
46	39	31	32	44	27	26	42	35	25

번외지만 global을 안쓰고 num을 파라미터로 받는 방식이 좋을듯

In [ ]:
# def clickBtn(num):
#     btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div[*]')

#     for btn in btns:
#         if btn.text == str(num):
#             btn.click()
#             return num + 1
#     return num

# num = 1
# while num <= 50:
#     num = clickBtn(num)


##### 색감테스트

https://zzzscore.com/color/

In [ ]:
# //*[@id="grid"]/div[1]
from selenium import webdriver
from pprint import pprint
import time

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/color/')
driver.implicitly_wait(300)

btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div')
print(len(btns))

230


정답을 구하기 위해서는  
1. 각 div의 rgba값 추출  
2. 추출한 rgba 중 값이 다른 하나가 정답  

selenium에서 추출한 요소는 value_of_css_property로 해당 요소의 css속성을 확인할 수 있음.

In [ ]:
//*[@id="grid"]/div[1]

In [ ]:
# //*[@id="grid"]/div[1]
from selenium import webdriver
from pprint import pprint
import time

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/color/')
driver.implicitly_wait(300)

btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div')

btns_rgba = [btn.value_of_css_property('background-color') for btn in btns]

pprint(btns_rgba)

['rgba(226, 101, 71, 1)',
 'rgba(226, 101, 71, 1)',
 'rgba(226, 101, 71, 1)',
 'rgba(193, 68, 38, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',
 'rgba(255, 0, 0, 1)',


btns_rgb에서 값을 찾기 위해서 collections 모듈의 counter 함수를 이용.  
Counter 함수는 값(value)과 개수를 딕셔너리(dict)로 반환

반환된 딕셔너리에서 value가 1인 것이 정답

In [ ]:
from collections import Counter

result = Counter(btns_rgba)
pprint(result)

Counter({'rgba(255, 0, 0, 1)': 226,
         'rgba(226, 101, 71, 1)': 3,
         'rgba(193, 68, 38, 1)': 1})


In [ ]:
for key, value in result.items():
    if value == 1:
        answer = key
        break
else:
    answer = None
    print('정답 없음')

In [ ]:
from selenium import webdriver
from pprint import pprint
import time
from collections import Counter

driver = webdriver.Chrome()
driver.get('https://zzzscore.com/color/')
driver.implicitly_wait(300)

btns = driver.find_elements(By.XPATH, '//*[@id="grid"]/div')

def analysis():
    btns_rgba = [btn.value_of_css_property('background-color') for btn in btns]

    result = Counter(btns_rgba)

    for key, value in result.items():
        if value == 1:
            answer = key
            break
    else:
        answer = None
        print('정답 없음')

    if answer:
        index = btns_rgba.index(answer)
        btns[index].click()

if __name__ == "__main__":
    start = time.time()
    while time.time() - start <= 60:
        analysis()

##### 브라우저 Driver에 옵션 주기

selenium을 통해 webdriver를 불러오기 전 여러가지 옵션을 줄 수 있음.  

- headless를 추가하면 크롬 창이 안 나옴  
- 접속 OS환경 지정  
- 화면 크기 지정  


In [ ]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920,1000')
service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service = service, options = options)

#### 파이썬으로 메일 보내기 smtplib, mime